In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn import tree
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import visualkeras
import tensorflow as tf
tf.data.experimental.enable_debug_mode()
from tensorflow.keras.callbacks import ModelCheckpoint
from tqdm.notebook import tqdm_notebook
from sklearn.tree import export_graphviz
from collections import defaultdict


In [5]:
def sequence_mining(team,opponent):
    combined_df = pd.read_csv('gen_sets/'+str(team)+'_runs.csv')
    combined_df = combined_df.replace({str(team):'same', str(opponent):'other'}, regex=True)
    
    df = pd.DataFrame()
    encoders = []

    for column in combined_df.columns[:-1]:
        le = LabelEncoder()
        encoders.append(le)
        df[column] = le.fit_transform(combined_df[column])

    df = pd.concat([df,combined_df.iloc[:,-1]], axis=1)

    undersample_len = len(df[df['class'] == 1])

    undersample_df = df[df['class'] == 0 ].sample(n=undersample_len, random_state=43)
    df = pd.concat([df[df['class'] == 1], undersample_df])

    print(df['class'].value_counts())
    
    run_events = {}
    run_max_counts = []

    for j, event in zip(range(12,112,11), range(10, 0, -1)):
        a = combined_df.iloc[:, -j:-1][combined_df['class'] == 1]

        # count occurrences of each row
        row_counts = defaultdict(int)
        for i in range(len(a)):
            row_tuple = tuple(a.iloc[i])
            row_counts[row_tuple] += 1

        # row with the maximum count
        max_count = 0
        mc_row = None
        for row, count in row_counts.items():
            if count > max_count:
                max_count = count
                mc_row = row

        # index of the row 
        mc_idx = a.apply(lambda row: tuple(row) == mc_row, axis=1).idxmax()


        print(f"Index: {mc_idx}, Max Count: {max_count}")
        run_events[event] = a.iloc[mc_idx]
        run_max_counts.append(max_count)
        print(f'{combined_df.iloc[mc_idx, -j:-1].to_frame().dropna().T}\n')

In [6]:
sequence_mining('home','away')

C:\Users\gsevr\AppData\Local\Temp\ipykernel_88852\811518203.py:2: DtypeWarning: Columns (104) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_df = pd.read_csv('gen_sets/'+str(team)+'_runs.csv')
C:\Users\gsevr\AppData\Local\Temp\ipykernel_88852\811518203.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column] = le.fit_transform(combined_df[column])
C:\Users\gsevr\AppData\Local\Temp\ipykernel_88852\811518203.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column] = le.fit_transf

class
1    18471
0    18471
Name: count, dtype: int64
Index: 16388, Max Count: 562
          Fouler10
16388  fouler_same

Index: 16418, Max Count: 108
           Rebounder9 ReboundType9     Fouler10
16418  rebounder_same    defensive  fouler_same

Index: 16446, Max Count: 53
            Rebounder8 ReboundType8 ShotDist9       Shooter9     Rebounder10   
16446  rebounder_other    defensive       3pt  shooter_other  rebounder_same  \

      ReboundType10  
16446     defensive  

Index: 16446, Max Count: 30
      ShotDist7      Shooter7       Rebounder8 ReboundType8 ShotDist9   
16446       3pt  shooter_same  rebounder_other    defensive       3pt  \

            Shooter9     Rebounder10 ReboundType10  
16446  shooter_other  rebounder_same     defensive  

Index: 16418, Max Count: 16
      ShotDist6      Shooter6       Rebounder7 ReboundType7 ShotDist8   
16418       3pt  shooter_same  rebounder_other    defensive       3pt  \

            Shooter8      Rebounder9 ReboundType9     Fouler1

In [7]:
sequence_mining('away', 'home')

C:\Users\gsevr\AppData\Local\Temp\ipykernel_88852\811518203.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column] = le.fit_transform(combined_df[column])
C:\Users\gsevr\AppData\Local\Temp\ipykernel_88852\811518203.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column] = le.fit_transform(combined_df[column])
C:\Users\gsevr\AppData\Local\Temp\ipykernel_88852\811518203.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perfor

class
1    16497
0    16497
Name: count, dtype: int64
Index: 3, Max Count: 4457
      Fouler10
3  fouler_same

Index: 6, Max Count: 978
  ShotDist9       Shooter9     Fouler10
6     close  shooter_other  fouler_same

Index: 17, Max Count: 423
         Rebounder8 ReboundType8 ShotDist9       Shooter9     Rebounder10   
17  rebounder_other    defensive       3pt  shooter_other  rebounder_same  \

   ReboundType10  
17     defensive  

Index: 171, Max Count: 194
    ShotDist7      Shooter7       Rebounder8 ReboundType8 ShotDist9   
171       3pt  shooter_same  rebounder_other    defensive       3pt  \

          Shooter9     Rebounder10 ReboundType10  
171  shooter_other  rebounder_same     defensive  

Index: 73, Max Count: 54
         Rebounder6 ReboundType6       Fouler7 FreeThrowShooter8   
73  rebounder_other    defensive  fouler_other          ft_other  \

   FreeThrowShooter9     Fouler10  
73          ft_other  fouler_same  

Index: 73, Max Count: 24
   ShotDist5      Shooter5    